In [1]:
import numpy as np
import torch
import joblib
from pathlib import Path

In [3]:
# device 선택 (GPU가 있으면 GPU, 없으면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [5]:
import torch.nn as nn

class FallLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=2, dropout=0.3, num_classes=2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                            batch_first=True, dropout=dropout, bidirectional=False)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        # x: (B, T, D)
        out, _ = self.lstm(x)       # out: (B, T, hidden)
        last = out[:, -1, :]        # 마지막 타임스텝을 사용 (B, hidden)
        logits = self.fc(last)      # (B, num_classes)
        return logits

In [7]:
BASE_DIR = Path.cwd().parent
ckpt_dir = BASE_DIR / "models"
best_ckpt = ckpt_dir / "best_fall_model.pth"

In [9]:
ck = torch.load(str(best_ckpt), map_location=device)
model = FallLSTM(input_size=132, hidden_size=128, num_layers=2).to(device)
model.load_state_dict(ck["model_state"])
model.eval()

FallLSTM(
  (lstm): LSTM(132, 128, num_layers=2, batch_first=True, dropout=0.3)
  (fc): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=64, out_features=2, bias=True)
  )
)

In [11]:
scaler_path = BASE_DIR / "src" / "scaler.save"  # 앞서 저장한 scaler
scaler = joblib.load(scaler_path) if scaler_path.exists() else None

C:\Users\un747\anaconda3\envs\fall\lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [12]:
DATA_PATH = BASE_DIR / "data" / "processed" / "npz" / "fall_dataset.npz"
# === npz 데이터 불러오기 ===
print("🔹 테스트 데이터 로드 중...")
data = np.load(DATA_PATH, allow_pickle=True)
X, Y = data["X"], data["Y"]

for _ in range(100):
    # === 한 샘플만 테스트 (낙상 or 비낙상 랜덤) ===
    idx = np.random.randint(0, len(X))
    sample = X[idx]
    label = Y[idx]
    
    # === 스케일 적용 ===
    # scaler는 (N*T, D)에 대해 fit했으므로, frame별로 변환
    sample_scaled = np.array([scaler.transform(frame.reshape(1, -1))[0] for frame in sample])
    
    # === 텐서 변환 ===
    x_tensor = torch.tensor(sample_scaled, dtype=torch.float32).unsqueeze(0).to(device)
    
    # === 예측 ===
    with torch.no_grad():
        logits = model(x_tensor)
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
        pred = int(np.argmax(probs))
    
    print("\n==== 🔍 예측 결과 ====")
    print(f"실제 라벨: {label}  →  예측 라벨: {pred}")
    print(f"클래스 확률: 낙상(1): {probs[1]:.4f}, 비낙상(0): {probs[0]:.4f}")
    print("======================")
    
    if pred == 1:
        print("🚨 낙상으로 감지되었습니다!")
    else:
        print("✅ 정상적인 행동입니다.")

🔹 테스트 데이터 로드 중...

==== 🔍 예측 결과 ====
실제 라벨: 0  →  예측 라벨: 0
클래스 확률: 낙상(1): 0.0016, 비낙상(0): 0.9984
✅ 정상적인 행동입니다.

==== 🔍 예측 결과 ====
실제 라벨: 0  →  예측 라벨: 0
클래스 확률: 낙상(1): 0.0002, 비낙상(0): 0.9998
✅ 정상적인 행동입니다.

==== 🔍 예측 결과 ====
실제 라벨: 1  →  예측 라벨: 1
클래스 확률: 낙상(1): 0.9999, 비낙상(0): 0.0001
🚨 낙상으로 감지되었습니다!

==== 🔍 예측 결과 ====
실제 라벨: 1  →  예측 라벨: 1
클래스 확률: 낙상(1): 0.9999, 비낙상(0): 0.0001
🚨 낙상으로 감지되었습니다!

==== 🔍 예측 결과 ====
실제 라벨: 0  →  예측 라벨: 0
클래스 확률: 낙상(1): 0.0002, 비낙상(0): 0.9998
✅ 정상적인 행동입니다.

==== 🔍 예측 결과 ====
실제 라벨: 1  →  예측 라벨: 1
클래스 확률: 낙상(1): 0.9999, 비낙상(0): 0.0001
🚨 낙상으로 감지되었습니다!

==== 🔍 예측 결과 ====
실제 라벨: 0  →  예측 라벨: 0
클래스 확률: 낙상(1): 0.0003, 비낙상(0): 0.9997
✅ 정상적인 행동입니다.

==== 🔍 예측 결과 ====
실제 라벨: 0  →  예측 라벨: 0
클래스 확률: 낙상(1): 0.0005, 비낙상(0): 0.9995
✅ 정상적인 행동입니다.

==== 🔍 예측 결과 ====
실제 라벨: 0  →  예측 라벨: 0
클래스 확률: 낙상(1): 0.0005, 비낙상(0): 0.9995
✅ 정상적인 행동입니다.

==== 🔍 예측 결과 ====
실제 라벨: 0  →  예측 라벨: 0
클래스 확률: 낙상(1): 0.0006, 비낙상(0): 0.9994
✅ 정상적인 행동입니다.

==== 🔍 예측 결과 ====
실제 라벨: 0  →  예측 라벨: 0
클래스 확

In [15]:
import cv2
import mediapipe as mp
VIDEO_PATH = BASE_DIR / "data" / "raw" / "LE2i" / "Home_01" / "Videos" / "video (1).avi"

In [29]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)
WINDOW_SIZE = 30

cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise RuntimeError(f"❌ 영상 파일을 열 수 없습니다: {VIDEO_PATH}")

keypoints_buffer = []

# ==============================
# 5️⃣ 프레임 루프
# ==============================
frameN = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    if results.pose_landmarks:
        # 33개 포인트 * (x,y,z)
        landmarks = results.pose_landmarks.landmark
        keypoints = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in landmarks]).flatten()
        keypoints_buffer.append(keypoints)

        if len(keypoints_buffer) >= WINDOW_SIZE:
            # 최근 WINDOW_SIZE 프레임 사용
            seq = np.stack(keypoints_buffer[-WINDOW_SIZE:], axis=0)  # (T, D)
            seq_scaled = scaler.transform(seq) if scaler else seq
            seq_scaled = np.expand_dims(seq_scaled, axis=0)  # (1, T, D)

            # 추론
            with torch.no_grad():
                x = torch.tensor(seq_scaled, dtype=torch.float32).to(device)
                logits = model(x)
                probs = torch.softmax(logits, dim=1)[0]
                pred = torch.argmax(probs).item()

            # 예측 결과 표시
            label_names = ["walk", "run", "jump", "sit", "stand"]  # 예시
            action = label_names[pred]
            cv2.putText(frame, f"Action: {action}", (20, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3, cv2.LINE_AA)
            if pred == 1:
                print(f"낙상 {frameN}")
            else:
                print("정상")

    #out.write(frame)
    cv2.imshow("Action Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frameN += 1
# ==============================
# 6️⃣ 종료 및 저장
# ==============================
cap.release()
#out.release()
cv2.destroyAllWindows()
print("✅ 영상 처리 완료! → output_result.mp4 저장됨")

C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


정상
정상


C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


정상
정상


C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


...

C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


낙상 258
낙상 259


C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


낙상 260
낙상 261
낙상 262
낙상 263
✅ 영상 처리 완료! → output_result.mp4 저장됨


C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\un747\anaconda3\envs\fall\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
